In [1]:
import os
import time
import pandas as pd
from github import Github
from credentials import username, password
from data_API import dataAPI

In [2]:
g = Github(username(), password())  # credentials for github login
repos = g.get_user().get_repos()  # iterable; all the repos associated with the profile incl. organizations

In [41]:
start = time.time()
api_data = dataAPI(repos, '2is50-2019-2020-homework-assignment-2-pair-')
end = time.time()
print("Time to complete:", end - start)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
Time to complete: 720.8968114852905


## Aggregate Data

In [19]:
df_data = pd.DataFrame(api_data)
all_unique_pairs = df_data['Pair'].unique()

### 1. Make dataset with commit size - additions & deletions

In [20]:
grouped_data = df_data.groupby(by=['Pair','Date','Author']).describe()

x = grouped_data['Additions'][['max']].reset_index()
x['Deletions'] = grouped_data['Deletions'][['max']].reset_index()['max']
x = x.rename(columns={'max': 'Additions'})
grouped_data = x

In [21]:
aggregated_list = []

for pair in all_unique_pairs:

    df_pair = grouped_data[grouped_data['Pair'] == pair]

    members = df_pair['Author'].unique()

    if len(members) == 2: # best scenario

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]
            m_2 = df_pair_date[df_pair_date['Author'] == members[1]]

            m1_additions = df_pair_date[df_pair_date['Author'] == members[0]]['Additions'].sum()
            m2_additions = df_pair_date[df_pair_date['Author'] == members[1]]['Additions'].sum()

            m1_deletions = df_pair_date[df_pair_date['Author'] == members[0]]['Deletions'].sum()
            m2_deletions = df_pair_date[df_pair_date['Author'] == members[1]]['Deletions'].sum()


            aggregated_list.append([pair, date, m1_additions, m2_additions, m1_deletions, m2_deletions])

    elif len(members) > 2: 

        custom_more = []

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]

            m1_additions = df_pair_date[df_pair_date['Author'] == members[0]]['Additions'].sum()
            m2_additions = df_pair_date[df_pair_date['Author'] == members[1]]['Additions'].sum()
            m2_additions += df_pair_date[df_pair_date['Author'] == members[2]]['Additions'].sum()


            m1_deletions = df_pair_date[df_pair_date['Author'] == members[0]]['Deletions'].sum()
            m2_deletions = df_pair_date[df_pair_date['Author'] == members[1]]['Deletions'].sum()
            m2_deletions += df_pair_date[df_pair_date['Author'] == members[2]]['Deletions'].sum()

            custom_more.append([pair, date, m1_additions, m2_additions, m1_deletions, m2_deletions])

        aggregated_list += custom_more 

    elif len(members) < 2: 

        custom_less = []

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]

            m1_additions = df_pair_date[df_pair_date['Author'] == members[0]]['Additions'].sum()
            m2_additions = 0

            m1_deletions = df_pair_date[df_pair_date['Author'] == members[0]]['Deletions'].sum()
            m2_deletions = 0

            custom_less.append([pair, date, m1_additions, m2_additions, m1_deletions, m2_deletions])

        aggregated_list += custom_less
        
        

In [22]:
cols = ['Pair', 'Date', 'M1_Additions', 'M2_Additions', 'M1_deletions', 'M2_deletions']
df_aggregated_member = pd.DataFrame(aggregated_list, columns=cols).sort_values(by='Pair')

### 2. Make Dataset with total of additions + deletions

In [24]:
df_total_size = df_aggregated_member#[['Pair', 'Date']]
df_total_size['Member_1'] = df_aggregated_member['M1_Additions'] + df_aggregated_member['M1_deletions']
df_total_size['Member_2'] = df_aggregated_member['M2_Additions'] + df_aggregated_member['M2_deletions']
df_total_size['Total'] = df_total_size['Member_1'] + df_total_size['Member_2'] 

In [32]:
df_total_size.to_csv("../data/commit_sizes_ha2.csv", index=False)

### 3. Make dataset with commits

In [26]:
grouped_data = df_data.groupby(by=['Pair','Date','Author']).describe()
grouped_data = grouped_data['Additions'][['count']].reset_index()

In [27]:
aggregated_list = []

for pair in all_unique_pairs:

    df_pair = grouped_data[grouped_data['Pair'] == pair]

    members = df_pair['Author'].unique()

    if len(members) == 2: # best scenario

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]['count'].sum()
            m_2 = df_pair_date[df_pair_date['Author'] == members[1]]['count'].sum()

            aggregated_list.append([pair, date, m_1, m_2])

    elif len(members) > 2:

        custom_more = []

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]['count'].sum()
            m_2 = df_pair_date[df_pair_date['Author'] == members[1]]['count'].sum()
            m_2 += df_pair_date[df_pair_date['Author'] == members[2]]['count'].sum()

            custom_more.append([pair, date, m_1, m_2])

        aggregated_list += custom_more 

    elif len(members) < 2:

        custom_less = []

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]['count'].sum()
            m_2 = 0

            custom_less.append([pair, date, m_1, m_2])

        aggregated_list += custom_less

In [28]:
cols = ['Pair', 'Date', 'Member_1', 'Member_2']
df_commits = pd.DataFrame(aggregated_list, columns=cols).sort_values(by='Pair')
df_commits['Total'] = df_commits['Member_1'] + df_commits['Member_2']

In [33]:
df_commits.to_csv("../data/commit_numbers_ha2.csv", index=False)

In [31]:
df_commits['Date'].unique()

array([datetime.date(2020, 5, 26), datetime.date(2020, 5, 27),
       datetime.date(2020, 6, 1), datetime.date(2020, 6, 2),
       datetime.date(2020, 6, 4), datetime.date(2020, 6, 12),
       datetime.date(2020, 6, 13), datetime.date(2020, 6, 14),
       datetime.date(2020, 6, 3), datetime.date(2020, 6, 7),
       datetime.date(2020, 6, 8), datetime.date(2020, 6, 9),
       datetime.date(2020, 6, 10), datetime.date(2020, 6, 11),
       datetime.date(2020, 5, 25), datetime.date(2020, 5, 19),
       datetime.date(2020, 6, 5), datetime.date(2020, 5, 20),
       datetime.date(2020, 5, 28), datetime.date(2020, 5, 22),
       datetime.date(2020, 5, 21), datetime.date(2020, 5, 23),
       datetime.date(2020, 5, 24), datetime.date(2020, 5, 29),
       datetime.date(2020, 5, 30), datetime.date(2020, 6, 6),
       datetime.date(2020, 5, 16), datetime.date(2020, 5, 17),
       datetime.date(2020, 5, 31), datetime.date(2020, 5, 18),
       datetime.date(2020, 5, 15)], dtype=object)

In [35]:
import datetime

In [39]:
df_commits[df_commits['Date'] == datetime.date(2020, 6, 14)]

,Pair,Date,Member_1,Member_2,Total
7,1,2020-06-14,4.0,0.0,4.0
155,23,2020-06-14,0.0,2.0,2.0
299,38,2020-06-14,2.0,0.0,2.0
320,40,2020-06-14,1.0,5.0,6.0
355,44,2020-06-14,2.0,2.0,4.0
363,45,2020-06-14,1.0,2.0,3.0
367,46,2020-06-14,9.0,9.0,18.0
375,47,2020-06-14,1.0,2.0,3.0
397,50,2020-06-14,4.0,5.0,9.0
416,52,2020-06-14,0.0,1.0,1.0
